## Transform multiple jsons from label me to coco format json

In [1]:
import json
import glob
import numpy as np
import os
import cv2
def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))
def create_folder(dirr):
    try:
        os.mkdir(dirr)
    except:
        pass

def extract_data(json_file,coco_json,c,ind,cats):
    with open(json_file) as f:
        file=json.load(f)
        width=file['imageWidth']
        height=file['imageHeight']
        path=file['imagePath'].split('\\')[-1]
        shapes=file['shapes']
        for shape in shapes:
            pts=[]
            xs=[]
            ys=[]
            bbox=[]
            ID=cats.index(shape['label'])+1
            for j in shape['points']:
                xs.append(float(j[0]))
                ys.append(float(j[1]))
                pts.append(float(j[0]))
                pts.append(float(j[1]))
            bbox=[min(xs),min(ys),max(xs),max(ys)]
            area=PolyArea(np.array(xs),np.array(ys))
            #print(area)
            ann_dict={"id": c,
              "iscrowd": 0,
              "image_id": ind+1,
              "category_id": ID,
              "segmentation":[pts],
              "bbox": bbox,
              "area": area
                     }
            coco_json["annotations"].append(ann_dict)
            c+=1
            
    return path,{"id": ind+1,
          "width": width,
          "height": height,
          "file_name": path}




def convert_files(dirr,cats,train_json,val_json,val_split,c=0):
    jsons=glob.glob(f'{dirr}/*.json')
    create_folder('images')
    create_folder('images/train')
    create_folder('images/val')

    for ind,i in enumerate(jsons):
        if ind<=(int(len(jsons)*(1-val_split))):
            img_dir,ret = extract_data(i,train_json,c=c,ind=ind,cats=cats)
            img=cv2.imread(f'{dirr}/{img_dir}')
            cv2.imwrite(f'images/train/{img_dir}',img)

            train_json['images'].append(ret)   

        else:
            img_dir,ret = extract_data(i,val_json,c=c,ind=ind,cats=cats)
            img=cv2.imread(f'{dirr}/{img_dir}')
            cv2.imwrite(f'images/val/{img_dir}',img)

            val_json['images'].append(ret)  

    with open('images/train/train.json','w') as fp:
        json.dump(train_json,fp)

    with open('images/val/val.json','w') as fp:
        json.dump(val_json,fp)

In [6]:
train_json=dict()
train_json['info']={ "description": "segmentation of stock data"}
train_json['images']=[]
train_json['annotations']=[]
train_json["categories"]= [{"id": 1,"name": "c"},{"id": 2,"name": "ihs"},{"id": 3,"name": "hs"}]
val_json=dict()
val_json['info']={ "description": "segmentation of stock data"}
val_json['images']=[]
val_json['annotations']=[]
val_json["categories"]= [{"id": 1,"name": "c"},{"id": 2,"name": "ihs"},{"id": 3,"name": "hs"}]
cats=['c','ihs','hs']

In [7]:
convert_files('test',cats,train_json,val_json,0.2)